# Семинар 3. Линейная регрессия

Подключение библиотек. 
$
\newcommand{\R}{\mathbb{R}}
\newcommand{\X}{\mathbb{X}}
\newcommand{\norm}[1]{\lVert #1 \rVert}
\newcommand{\abs}[1]{\left| #1 \right|}
\newcommand{\E}{\mathbb{E}}
\newcommand{\D}{\mathbb{D}}
\renewcommand{\Prob}{\mathbb{P}}
\renewcommand{\le}{\leqslant}
\renewcommand{\ge}{\geqslant}
$

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as wg

plt.style.use('ggplot')

# 1. Линейная регрессия из библиотеки sklearn

Для демонстрации воспользуемся датасетом __вес-рост__ ([ссылка](https://www.kaggle.com/mustafaali96/weight-height)):
* 10000 наблюдений, 5000 мужчин и 5000 женщин. Признаки:
    * Рост в дюймах.
    * Вес в фунтах.
    * Пол (Male/ Female).

In [ ]:
# прочитать файл weight-height.csv
df = pd.read_csv('weight-height.csv')

print('Число наблюдений:', df.shape[0])
df.head()

__Задача:__ Переведем вес и рост в привычные единицы измерения по формулам:
* 1 фунт = 0.453592 кг.
* 1 дюйм = 2.54 см.

In [ ]:
# перевести в кг и см
df['Height_cm']

In [ ]:
df.mean()

Совместное распределение данных:

In [ ]:
# график совместного распределения
sns.jointplot(data=df, x='Height_cm', y='Weight_kg')

In [ ]:
sns.jointplot(data=df, x='Height_cm', y='Weight_kg', hue='Gender')

In [ ]:
# приготовить X и y
X = df[['Height_cm']].values
y = df['Weight_kg'].values

print('X:', X.shape)
print('y:', y.shape)

## Классическая линейная регрессия

Пусть объект описывается $d$ признаками $(x_1, \ldots, x_d)$ и нужно предсказать ответ $y$.

__Линейная регрессия:__ $$ \hat{y} \sim \theta_0 + \sum\limits_{k=1}^d \theta_k x_k, $$

* $\hat{y}$ -- прогнозное значение.
* $x_k, k=\overline{1,n}$ -- значения признаков объекта.
* $\theta_k, k=\overline{1,n}$ -- параметры модели.

Векторная запись: $$\hat{y} = x^T \theta. $$

__Примечание:__ В этой записи считаем, что $x_0 = 1$.

### Обучение: 
Пусть имеется матрица признаков $X \in \R^{n x d}$ и вектор ответов $Y \in \R^n$. 

__Метод наименьших квадратов:__ $$Q(\theta) = ||Y - X\theta||^2 = \sum\limits_{k=1}^n (Y_k - X_k^T \theta)^2 \to \min\limits_\theta.$$

Оптимизация:
* Аналитическое решение: $$\hat{\theta} = (X^T X)^{-1} X^T Y.$$
* Итерационные методы: 
    * _градиентный спуск (GD):_ $$\theta^{(k+1)} = \theta^{(k)} - \alpha \cdot \nabla Q(\theta^{(k)}).$$
    * _стохастический градиентный спуск (SGD):_ $$\theta^{(k+1)} = \theta^{(k)} - \alpha \cdot \nabla_i Q(\theta^{(k)}), $$ где градиент берется $\nabla_i$ берется по наблюдению со случайным индексом $i$.
    * _mini-batch стохастический градиентный спуск (Mini-batch SGD)._

In [ ]:
# обучим линейную модель
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)
model

__Задача:__ отобразить набор данных через scatter-график и вывести прямую предсказаний линейной модели.

In [ ]:
# код

# 2. Работа с категориальными признаками.

Заметим, что в наших данных имеется категориальный признак _Gender_.

In [ ]:
df.Gender.value_counts()

In [ ]:
sns.jointplot(data=df, x='Height_cm', y='Weight_kg', hue='Gender')

__Задача:__ Занумеруем и превратим текстовые признаки в числовые значения:

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df.Gender)

encoder.transform(df.Gender)

In [ ]:
encoder = LabelEncoder()
encoder.fit(['car', 'bike', 'ship', 'chair'])
encoder.classes_

In [ ]:
encoder.transform(['car', 'car', 'bike', 'ship', 'ship'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

codes = np.array(['car', 'car', 'bike', 'ship', 'ship']).reshape(-1, 1)
onehot = OneHotEncoder(sparse=False, drop='first')
onehot.fit(codes)
onehot.transform(codes)

In [ ]:
df['Gender_code'] = df.Gender.map({'Female': 0, 'Male': 1})
df.Gender_code.value_counts()

__Задача:__ Обученить линейную модель с двумя признаками.

In [ ]:
X = df[['Height_cm', 'Gender_code']].values
y = df['Weight_kg'].values

print(X.shape, y.shape)

In [ ]:
model = LinearRegression()
model.fit(X, y)
model

__Задача:__ визуализировать предсказание модели для парней и девушек отдельно.

In [ ]:
x_plt = np.linspace(130, 210, 1024)

X1 = np.column_stack([x_plt, np.zeros_like(x_plt)])
X2 = np.column_stack([x_plt, np.ones_like(x_plt)])

print(X1[:5])
print(X2[:5])

male_pred = model.predict(X2)
female_pred = model.predict(X1)

In [ ]:
plt.figure(figsize=(12, 8))
# код
plt.show()

### Интерпретация модели

In [ ]:
print('Коэффициенты:', model.coef_)

In [ ]:
print('Theta_0:', model.intercept_)

In [ ]:
features = np.array([[192, 1]])
print(features.shape)

model.predict(features)

In [ ]:
# демонстрация
out = wg.Output()

def demo(height, gender):
    gender = 1 if gender == 'Male' else 0
    with out:
        out.clear_output(wait=True)
        pred = model.predict([[height, gender]])
        print('Прогноз Вашего веса при заданных параметрах:', pred)
    return

wg.interact(demo,
    height=wg.FloatSlider(min=120, max=220, value=170, step=1),
    gender=wg.SelectionSlider(options=['Male', 'Female'], value='Male'),
    continuous_update=True
)
display(out)

__Задача:__ Добавить в модель переменную `Gender_Code * Height_cm`. Дать интерпретацию, выписать формулы.

In [ ]:
# код

# 3. Нелинейная линейная регрессия

Как добавить нелинейности в линейную модель?

In [ ]:
# задать функцию func = sin x
def func(x):
    # код

In [ ]:
# сгенерируем выборку с шумом из функции sin x

x_low, x_high = -np.pi, np.pi

N = 32
X = np.linspace(x_low, x_high, N)
y = func(X) + np.random.normal(0, 0.2, N)

In [ ]:
x_plt = np.linspace(x_low, x_high, 1024)
y_plt = func(x_plt)

plt.figure(0, figsize=(12, 5))
plt.scatter(X, y, color='blue')
plt.plot(x_plt, y_plt, linestyle='--', color='green')
plt.show()

__Полиномиальная регрессия:__ $$ y \sim \theta_0 + \sum\limits_{k=1}^p \theta_k x^k.$$

__Задача:__ По входному вектору $x$ выдать матрицу, где в столбцах стоят степени $x$: $$ x \to (x, x^2, \ldots, x^p).$$

In [ ]:
P = 5

Способ 1:

In [ ]:
# круто, но сложно
Xp = np.power(X.reshape(-1, 1), np.arange(1, P + 1).reshape(1, -1))
print(Xp.shape)
Xp[:2]

Способ 2:

In [ ]:
def polynom(x, p):
    powers = []
    for k in range(1, p+1):
        powers.append(x ** k)
    return np.column_stack(powers)

In [ ]:
Xp = polynom(X, P)
print(Xp.shape)
Xp[:2]

Способ 3 (рекомендуемый):

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=5, include_bias=False)
Xp = poly.fit_transform(X.reshape(-1, 1))
print(Xp.shape)
Xp[:2]

### Работа с числовыми признаками

Преобразования числовых признаков:
* нормализация: $$\tilde{x}_k = \frac{x_k - \E x_k}{\sqrt{\D x_k}}.$$
* минмакс-преобразование: $$\tilde{x}_k = \frac{x_k - \min{x_k}}{\max{x_k} - \min{x_k}}.$$

In [ ]:
print(Xp.mean(axis=0))
print(Xp.std(axis=0))

In [ ]:
# нормализация признаков
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(Xp)

Xps = scaler.transform(Xp)

In [ ]:
# новые средние
print(np.isclose(Xps.mean(axis=0), 0))
print(Xps.std(axis=0))

## Пример: полиномиальная регрессия

In [ ]:
# учим модель
model = LinearRegression()
model.fit(Xps, y)
model

In [ ]:
# предсказываем новые данные

x_new = np.linspace(x_low, x_high, 1024)
# x_new = np.linspace(0.2, 7, 1024)

x_new.shape

In [ ]:
X_new = polynom(x_new, P)
X_new.shape

In [ ]:
print(X_new.mean(axis=0))
print(X_new.std(axis=0))

X_new = scaler.transform(X_new)

print(X_new.mean(axis=0))
print(X_new.std(axis=0))

In [ ]:
y_new = model.predict(X_new)
y_new[:5]

In [ ]:
y_old = model.predict(Xps)

In [ ]:
plt.figure(0, figsize=(12, 5))
plt.scatter(X, y, color='blue')
plt.plot(x_new, func(x_new), color='green', linestyle='--', label='True func')
plt.plot(x_new, y_new, color='purple', label='Polynomial Regression')
plt.legend()
plt.title('y ~ log(x)')
plt.show()

# Демонстрация переобучения

In [ ]:
N = 32

X = np.linspace(x_low, x_high, N)
y = func(X) + np.random.normal(0, 0.2, size=N)

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
out = wg.Output()

def demo(p, fix_ylim):
    Xp = polynom(X, p)
    
    # scaler
    scaler = StandardScaler().fit(Xp)    
    Xps = scaler.transform(Xp)
    
    # model
    model = LinearRegression() #
#     model = Ridge(alpha=1)
    model.fit(Xps, y)
    
    # prediction
    x_new = np.linspace(x_low, x_high, 1024)

    X_new = polynom(x_new, p)
    X_new = scaler.transform(X_new)
    y_new = model.predict(X_new)
    
    with out:
        out.clear_output(wait=True)
        plt.figure(0, figsize=(12, 5))
        plt.scatter(X, y, color='blue')
        plt.plot(x_new, func(x_new), color='green', linestyle='--', label='True func')
        plt.plot(x_new, y_new, color='purple', label=f'p={p}')
        plt.legend()
        plt.title('y ~ log(x)')
        if fix_ylim:
            plt.ylim(-2, 3)
        plt.show()
        print(model.coef_)
        print(model.intercept_)
    
wg.interact(demo,
    p=wg.IntSlider(min=1, max=X.shape[0] + 20, value=1),
    fix_ylim=wg.Checkbox(description='Fix ylim', value=False),
    continuous_update=True
)
display(out)